WeGo Public Transit
-----

WeGo Public Transit is a public transit system serving the Greater Nashville and Davidson County area. WeGo provides local and regional bus routes, the WeGo Star train service connecting Lebanon to downtown Nashville, along with several other transit services.

In this project, your goal will be to analyze data from WeGo's bus routes and attempt to identify intersections and other locations which experience frequent delays. This analysis can look by route, by direction of travel, and by day or day of the week.

You can also consider time of day in your investigation (for example, comparing the AM and PM rush on Monday through Friday).

You can obtain the data for this project through Swiftly. You will be provided with an API key to access this data. The API that you will be using is the Speed Map API, which provides average speed information for a single route and direction over a given date range.

Note that the API can return either speed data at the stop segment level or return high resolution data, which provides information for stop segments of approximately 25 meters along with more detailed speed and duration information.

Please take the time to carefully read the API documentation so that you are familiar with the query parameter option and understand the response data.

You can get more information about these routes, such as route numbers and stop information, using the Agency Routes API. A user-friendly map showing all routes is available at https://www.wegotransit.com/ride/maps-schedules/bus/.

If time allows, investigate all routes, but start with the high ridership and frequent routes:

3: West End
7: Hillsboro
22: Bordeaux
23: Dickerson Pike
50: Charlotte Pike
52: Nolensville Pike
55: Murfreesboro Pike
56: Gallatin Pike
Bottlenecks - constant/irregular (constant) ignoring bus stop sections Finding downtown vs suburbs bottlenecks 1st Quartile of each route for constantDelayed data

Focus on Inbound/Outbound (inbound)

Time categories - (6am - 9am AM Peak) 9am-230pm Off Peak 230pm-6pm PM Peak 6pm-6am Overnight Day of Interest Saturday Sunday Holidays* (M/F) (Tues-Thurs) (5/25-7/26) (1/25-5/24)

API access
----------

Agency Info: agency_details, routes, schedule

Speed-map: Check all, speed-map

In [ ]:
import requests
import json
from shapely.geometry import LineString
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('../data/WeGo.json') as fi:
    credentials = json.load(fi)

In [3]:
Authorization = credentials['Authorization']

In [4]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params_geo3 = {
    'direction': '1',
    'startDate': '01-07-2022',
    'endDate': '01-26-2022',
    'daysOfWeek': '1,2,3,4,5',
    'beginTime': '06:00',
    'endTime': '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'format': 'geojson',
    'resolution': 'hiRes'
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

response_geo3 = requests.get(URL, params = params_geo3, headers = headers)
response_geo3

# dig into json to get list of line-segment dicts
segments3 = response_geo3.json()['features']

# convert to GeoDataFrame the quick way
fullweekgeo = gpd.GeoDataFrame.from_features(segments3)

fullweekgeo

In [5]:
response_geo3.json()

NameError: name 'params' is not defined

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '2,3,4',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '2,3,4',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

junjult = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

junjult = junjult[junjult['countForSegment'] > 40]

junjult

M-F inbound df, sorted by 30 day chunks Jan 25-July26
-----

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

janfebinbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

janfebinbound = janfebinbound.loc[janfebinbound['countForSegment'] > janfebinbound['countForSegment'].max()/2]

janfebinbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

febmarinbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

febmarinbound = febmarinbound.loc[febmarinbound['countForSegment'] > febmarinbound['countForSegment'].max()/2]

febmarinbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

maraprinbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

maraprinbound = maraprinbound.loc[maraprinbound['countForSegment'] > maraprinbound['countForSegment'].max()/2]

maraprinbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

aprmayinbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

aprmayinbound = aprmayinbound.loc[aprmayinbound['countForSegment'] > aprmayinbound['countForSegment'].max()/2]

aprmayinbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mayjuninbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

mayjuninbound = mayjuninbound.loc[mayjuninbound['countForSegment'] > mayjuninbound['countForSegment'].max()/2]

mayjuninbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '1',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

junjulinbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

junjulinbound = junjulinbound.loc[junjulinbound['countForSegment'] > junjulinbound['countForSegment'].max()/2]

junjulinbound

M-F outbound df, sorted by 30 day chunks Jan 25-July26
-----

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '01-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '02-24-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

janfeboutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

janfeboutbound = janfeboutbound.loc[janfeboutbound['countForSegment'] > janfeboutbound['countForSegment'].max()/2]

janfeboutbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '02-25-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '03-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

febmaroutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

febmaroutbound = febmaroutbound.loc[febmaroutbound['countForSegment'] > febmaroutbound['countForSegment'].max()/2]

febmaroutbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '03-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '04-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

maraproutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

maraproutbound = maraproutbound.loc[maraproutbound['countForSegment'] > maraproutbound['countForSegment'].max()/2]

maraproutbound

In [ ]:

URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

aprmayoutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

aprmayoutbound = aprmayoutbound.loc[aprmayoutbound['countForSegment'] > aprmayoutbound['countForSegment'].max()/2]

aprmayoutbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '05-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '06-25-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

mayjunoutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

mayjunoutbound = mayjunoutbound.loc[mayjunoutbound['countForSegment'] > mayjunoutbound['countForSegment'].max()/2]

mayjunoutbound

In [ ]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/50'

params = {
    'direction': '0',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'format' : 'geojson',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

params1 = {
    'direction': '0',
    'startDate': '06-26-2022',
    'beginTime' : '06:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '07-26-2022',
    'endTime' : '09:00',
    'excludeDates': '01-17-2022,02-18-2022,03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022,04-14-2022,04-15-2022,05-30-2022,07-04-2022',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

junjuloutbound = concat_frames(requests.get(URL, params = params, headers = headers), requests.get(URL, params = params1, headers = headers))

junjuloutbound = junjuloutbound.loc[junjuloutbound['countForSegment'] > junjuloutbound['countForSegment'].max()/2]

junjuloutbound